## 매수시간(15시35분~15시45분) 이고 금일 매수내역이 없으면 매수한다.
1. 매수시간이면
    1. 금일 매수내역이 없으면
        1. 매수여부, 매수강도로 매수수량 산출
        1. 매수주문
        1. DB에 기록

## 매도시간(08시30분~15시35분) 이고 전일 잔고가 청산이 되지않았고, 계좌 잔고가 남아있으면 청산한다.
1. 매도시간이면
    1. DB상 전일잔고가 +인가
        1. 계좌잔고가 +인가
            1. 계좌잔고만큼 매도 주문
            1. DB에 매도기록



In [5]:
import sys
import win32com.client
import time
import sqlite3
import os
from datetime import datetime, timedelta

global fd
fd = open("./trade.log", 'a', encoding="MS949")  ## 3)

def log(msg):
    fd.write((datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " : " + msg + "\n"))
    print((datetime.now().strftime("%Y-%m-%d %H:%M:%S")  + " : " +  msg ))
    fd.flush()

conn = sqlite3.connect('example.db')
c = conn.cursor()
symbol = '106Q9'
line = ''

while True:
    
    today = datetime.now().strftime("%Y-%m-%d")
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

    # log("today : " + today)
    # log("yesterday : " + yesterday)

    nowtime = datetime.now().strftime("%H:%M:%S")
    # log(nowtime)

    if nowtime > '15:35:30' and nowtime < '15:45:59' :
        log( "매수시간")

        rowcnt = 0
        for r in c.execute("SELECT sum(수량) FROM trans WHERE 종목코드 = '%s' and substr(거래일시,1,10) = '%s'" % (symbol , today)):
            posSum = r[0]
            rowcnt += 1

        log('당일잔고(DB)) : %f' % posSum)
        if posSum > 0 :
            log("안삼")
        else :
            log("매수여부, 주문강도로 매수수량 산출")
            log("샀으면 DB에 기록")

    # elif nowtime > '00:00:00' and nowtime < '23:59:59' :
    elif nowtime > '08:31:00' and nowtime < '15:35:59' :
        log("매도시간")

        rowcnt = 0
        for r in c.execute("SELECT sum(수량) FROM trans WHERE 종목코드 = '%s'" % (symbol)):
            posSum = r[0]
            rowcnt += 1

        if posSum == None :
            posSum = 0.0

            
        log('전일잔고(DB) : %f' % posSum)            
        
        if posSum > 0 :
            log("계좌잔고만큼 매도 주문")
            log("DB에 매도기록")
            c.execute("INSERT INTO trans(종목코드, 거래일시, 종류, 수량, 가격) VALUES (?,?,?,?,?)" , ('106Q9', datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '매도', posSum * -1, 2000000 ) )
            conn.commit()

        else :
            log("팔게 없다")
        


        # for row in c.execute("SELECT sum(수량) FROM trans WHERE 종목코드 = '%s' and substr(거래일시,1,10) = '%s'" % (symbol , today)):

    time.sleep(3)
    
    
    # text_str = file.read()  ## 4)
    # print(text_str)  

file.close()  ## 5)
c.close()




2020-07-30 11:48:44 : 매도시간
2020-07-30 11:48:44 : 전일잔고(DB) : 0.000000
2020-07-30 11:48:44 : 팔게 없다
2020-07-30 11:48:47 : 매도시간
2020-07-30 11:48:47 : 전일잔고(DB) : 0.000000
2020-07-30 11:48:47 : 팔게 없다
2020-07-30 11:48:50 : 매도시간
2020-07-30 11:48:50 : 전일잔고(DB) : 0.000000
2020-07-30 11:48:50 : 팔게 없다


KeyboardInterrupt: 